In [5]:
#To be able to run this code, all the libraries below need to be downloaded.
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
from transformers import pipeline

#Reading the text file into
df = pd.read_csv('texts.csv')

#Pre-processing the data
#Split the file into texts and labels
#Since the dataset of texts was made by me. I have labeled the different texts as 1, if it is a text about food poisoning, and 0 otherwise.
text = df.drop('label', axis=1)
labels = df['label']

#Vectorize the data
vectorizer = CountVectorizer()
vectorizedText = vectorizer.fit_transform(X['texts'])

#Creating the clusters
kmeans = KMeans(n_clusters=2, n_init = 10)
kmeans.fit(vectorizedText)

#Assigning texts to the clusters
clusterValues = kmeans.predict(vectorizedText)

#The clusters are labeled 0 or 1(since there are only 2 clusters) randomly, so I create a new list with labels similar to the ones in the data file
#This is helpful when trying to get the accuracy score of the prediction model
correctedClusterValues = []
poisoning = 0
for i in range(len(clusterValues)):
    if "poisoning" in text['texts'][i]:
        poisoning = clusterValues[i]
        break

if poisoning != 1:
    for i in clusterValues:
        if i == 1:
            correctedClusterValues.append(0)
        elif i == 0:
            correctedClusterValues.append(1)
else:
    correctedClusterValues = clusterValues

#Finding the accuracy score
accuracy = accuracy_score(y, y_pred_new)
print('Accuracy:', accuracy)

#Getting the texts which are about food poisoning
poisoning_text = ""
for i in range(len(correctedClusterValues)):
    if correctedClusterValues[i] == 1:
        poisoning_text += " " + text['texts'][i]

#Summarizing the text using the Transformers model
model_name = "google-t5/t5-small"
summarizer = pipeline('summarization', model = model_name)
summary = summarizer(poisoning_text, max_length = 200, min_length = 30, do_sample = False)
summaryText = summary[0]['summary_text']

print(summaryText)

Accuracy: 1.0


'food poisoning outbreak grips sub-urban Bangalore, prompting closures and investigations . residents urge vigilance after a string of incidents linked to a local restaurant . reports of poisoning spark health scare among residents .'